<a href="https://colab.research.google.com/github/fjasic/duusaiuv/blob/main/vezba2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Training.zip

--2021-02-02 14:18:44--  https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Training.zip
Resolving btsd.ethz.ch (btsd.ethz.ch)... 129.132.52.185, 2001:67c:10ec:36c2::b9
Connecting to btsd.ethz.ch (btsd.ethz.ch)|129.132.52.185|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174298785 (166M) [application/zip]
Saving to: ‘BelgiumTSC_Training.zip’

BelgiumTSC_Training 100%[===================>] 166.22M   739KB/s    in 5m 18s  

2021-02-02 14:24:02 (536 KB/s) - ‘BelgiumTSC_Training.zip’ saved [174298785/174298785]



In [2]:
!wget https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Testing.zip

--2021-02-02 14:24:04--  https://btsd.ethz.ch/shareddata/BelgiumTSC/BelgiumTSC_Testing.zip
Resolving btsd.ethz.ch (btsd.ethz.ch)... 129.132.52.185, 2001:67c:10ec:36c2::b9
Connecting to btsd.ethz.ch (btsd.ethz.ch)|129.132.52.185|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 77877833 (74M) [application/zip]
Saving to: ‘BelgiumTSC_Testing.zip’

BelgiumTSC_Testing. 100%[===================>]  74.27M   602KB/s    in 1m 52s  

2021-02-02 14:25:57 (677 KB/s) - ‘BelgiumTSC_Testing.zip’ saved [77877833/77877833]



In [3]:
!unzip BelgiumTSC_Training.zip
!unzip BelgiumTSC_Testing.zip

Streaming output truncated to the last 5000 lines.
  inflating: Training/00036/01285_00000.ppm  
  inflating: Training/00036/01285_00001.ppm  
  inflating: Training/00036/01285_00002.ppm  
  inflating: Training/00036/01457_00000.ppm  
  inflating: Training/00036/01457_00001.ppm  
  inflating: Training/00036/01457_00002.ppm  
  inflating: Training/00036/01606_00000.ppm  
  inflating: Training/00036/01606_00001.ppm  
  inflating: Training/00036/01606_00002.ppm  
  inflating: Training/00036/01687_00000.ppm  
  inflating: Training/00036/01687_00001.ppm  
  inflating: Training/00036/01687_00002.ppm  
  inflating: Training/00036/GT-00036.csv  
   creating: Training/00037/
  inflating: Training/00037/00099_00000.ppm  
  inflating: Training/00037/00099_00001.ppm  
  inflating: Training/00037/00099_00002.ppm  
  inflating: Training/00037/00100_00000.ppm  
  inflating: Training/00037/00163_00000.ppm  
  inflating: Training/00037/00163_00001.ppm  
  inflating: Training/00037/00163_00002.ppm  
  i

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [4]:
# Vežba 2 - potpuno povezani slojevi
from __future__ import division, print_function, absolute_import

import numpy as np # linearna algebra
import pandas as pd # obrada podataka, csv datoteka (npr. pd.read_csv)

import matplotlib
import matplotlib.image as mpimg
matplotlib.use('agg')
import matplotlib.pyplot as plt

import skimage.data
import skimage.transform

import tensorflow as tf

import random
import os

print(tf.__version__)
# neuralna mreža sa 1 slojem od 62 softmax neurona
#
# · · · · · · · · · ·       (ulatni podaci, jednodimenzionalni)  X [batch, 3072]  # 3072 = 32 * 32 * 3
# \x/x\x/x\x/x\x/x\x/    -- potpuno poveyani sloj (softmax)      W [3072, 62]     B[62]
#   · · · · · · · ·                                              Y [batch, 62]

# Model je:
#
# Y = softmax( X * W + b)
#              X: matrica 183 'flattened' (1d) slike 32x32x3 elementa,( 183 slike u batch-u)
#              W: matrica koeficijenata od 3072 reda i 62 kolone
#              B: vektor bias koeficijenata dimenzije 62
#              softmax(matrica) primenjuje softmax na svaki red
#              softmax(red) primenjuje eksponent na svaku vrednost, zatim deli normalizovanom vrednošću svakog reda 
#              Y: izlazna matrica sa 183 reda i 62 kolone

1.15.2


In [18]:
# Putanje do trening i test skupova podataka.
train_data_dir = os.path.join("Training")
test_data_dir = os.path.join("Testing")
print(train_data_dir)
###########################################################################################
# Pomoćna funkcija za učitavanje skupa podataka
# load_data - učitava skup podataka i vraća dve liste: 
# - images: lista Numpy nizova, svaki predstavlja sliku.
# - labels: lista brojeva koji predstavljaju labele slika.
###########################################################################################
def load_data(data_dir):
    # Prikupljanje svih poddirektorijume direktorijuma data_dir. Svaki od njih predstavlja labelu.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Iteriranje kroz direktorijume (sa imenima labela) i prikupljanje podataka u dve liste: labele i slike.
    labels = []
    images = []
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f) 
                      for f in os.listdir(label_dir) if f.endswith(".ppm")]
        # Za svaku labelu, učitavanje pripadajućih slika i dodavanje u listu slika.
        # I dodavanje labele (t.j. imena direktorijuma) u listu labela.
        for f in file_names:
            images.append(mpimg.imread(f))
            labels.append(int(d))
    return images, labels

###########################################################################################
# Učitavanje i predobrada trening skupa podataka
###########################################################################################
# Učitavanje trening skupa podataka.
images, labels = load_data(train_data_dir)

print("\nTrening skup podataka\n-Broj klasa/labela: {0}\n-Ukupan broj slika: {1}".format(len(set(labels)), len(images)))

# Za prvih 5 slika prikaz rezolucije slika (shape), kao i minimalne i maksimalne vrednosti niza (vrednost komponenti boja)
for image in images[:5]:
    print("\nOriginalni dataset: Oblik: {0}, min: {1}, max: {2}".format(image.shape, image.min(), image.max()))

# Skaliranje slika na istu rezoluciju (WxH -> 32x32)
scaled_imgs = [skimage.transform.resize(image, (32, 32), mode='constant')
                for image in images]

# Za prvih 5 slika prikaz rezolucije slika (shape), kao i minimalne i maksimalne vrednosti niza (vrednost komponenti boja)
for image in scaled_imgs[:5]:
    print("\nSkalirani dataset: Oblik: {0}, min: {1}, max: {2}".format(image.shape, image.min(), image.max()))

# Promena formata skladištenja: lista -> niz
labels_a = np.array(labels)         # lista -> (4575,)
images_a = np.array(scaled_imgs)    # lista -> (4575, 32, 32, 3)
# Prikaz 
print("\nRezultat predobrade:\n-Oblik labela: ", labels_a.shape, "\n-Oblik slika: ", images_a.shape)

Training

Trening skup podataka
-Broj klasa/labela: 62
-Ukupan broj slika: 4575

Originalni dataset: Oblik: (172, 178, 3), min: 0, max: 154

Originalni dataset: Oblik: (142, 159, 3), min: 4, max: 255

Originalni dataset: Oblik: (103, 110, 3), min: 0, max: 255

Originalni dataset: Oblik: (176, 191, 3), min: 0, max: 246

Originalni dataset: Oblik: (120, 131, 3), min: 3, max: 234

Skalirani dataset: Oblik: (32, 32, 3), min: 0.0392156862745098, max: 0.5843137254901961

Skalirani dataset: Oblik: (32, 32, 3), min: 0.0737132352941179, max: 0.9447744332107844

Skalirani dataset: Oblik: (32, 32, 3), min: 0.06496055453431364, max: 0.996078431372549

Skalirani dataset: Oblik: (32, 32, 3), min: 0.09019607843137255, max: 0.8990196078431374

Skalirani dataset: Oblik: (32, 32, 3), min: 0.0872625612745094, max: 0.8828431372549019

Rezultat predobrade:
-Oblik labela:  (4575,) 
-Oblik slika:  (4575, 32, 32, 3)


In [35]:
###########################################################################################
# Kreiranje modela
###########################################################################################
num_inputs = 3072      # 32*32*3
num_classes = 62       # broj neurona izlaznog sloja = broj klasa
num_hidden_1 = 200     # broj neurona skrivenog sloja 1
num_hidden_2 = 100     # broj neurona skrivenog sloja 2

# Dodavanje 2 nova skrivena sloja
W1 = tf.Variable(tf.truncated_normal(shape=[num_inputs, num_hidden_1], stddev=0.1), name = "W1")      
B1 = tf.Variable(tf.ones([num_hidden_1])/10, name = "B1") 

W2 = tf.Variable(tf.truncated_normal(shape=[num_hidden_1, num_hidden_2], stddev=0.1), name = "W2")      
B2 = tf.Variable(tf.ones([num_hidden_2])/10, name = "B2") 

# Izlazni sloj
W3 = tf.Variable(tf.truncated_normal(shape=[num_hidden_2, num_classes], stddev=0.1), name = "W3")      
B3 = tf.Variable(tf.ones([num_classes])/10, name = "B3") 

# Rezervisane varijable za slike i labele.
images_ph = tf.placeholder(tf.float32, [None, 32, 32, 3])
labels_ph = tf.placeholder(tf.int32, [None])

# Promena formata skladištenja slika 
images_flat = tf.contrib.layers.flatten(images_ph)
sloj_1 = tf.nn.relu(tf.matmul(images_flat, W1) + B1)
sloj_2 = tf.nn.relu(tf.matmul(sloj_1, W2) + B2)
logits = tf.matmul(sloj_2, W3) + B3
Y = tf.nn.softmax(logits)
# Određivanje labele na osnovu verovatnoća izlaznog sloja
predicted_classes = tf.argmax(Y, 1)

# Definisanje funcije cene
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels_ph))

# Postavljanje optimizacione funkcije i brzine učenja
train = tf.train.MomentumOptimizer(learning_rate=0.0005, momentum=0.9, use_nesterov=True).minimize(loss)

# Operacija inicijalizacije varijabli
init = tf.global_variables_initializer()


In [36]:
###########################################################################################
# Proces treniranja
###########################################################################################

# Podešavanje parametara 
num_epochs = 51   # broj epoha
num_batches = 25  # broj batch-eva u okviru  epohe
b_size = 183      # broj slika u batch-u

# Kreiranje sesije
session = tf.Session()
session.run(init)

# Za svaku epohu
for i in range(num_epochs):
    # I svaki batch u epohi
    for j in range(num_batches):
        _, loss_value = session.run([train, loss], 
                                    feed_dict={images_ph: images_a[j*b_size:(j+1)*b_size], labels_ph: labels_a[j*b_size:(j+1)*b_size]})
                                    
    if i % 10 == 0:
        # Izvršavanje "predicted_classes" čvora
        predicted = session.run([predicted_classes], feed_dict={images_ph: scaled_imgs})[0]

        # Izračunavanje tačnosti
        match_count = sum([int(y == y_) for y, y_ in zip(labels, predicted)])
        accuracy = match_count / len(labels)
        print("Trening podaci: Epoha {0}: Mera gubitka: {1}, Tačnost: {2}".format(i, loss_value, accuracy))


Trening podaci: Epoha 0: Mera gubitka: 4.940857410430908, Tačnost: 0.04437158469945355
Trening podaci: Epoha 10: Mera gubitka: 4.075752258300781, Tačnost: 0.4037158469945355
Trening podaci: Epoha 20: Mera gubitka: 3.414029121398926, Tačnost: 0.5486338797814208
Trening podaci: Epoha 30: Mera gubitka: 2.8865036964416504, Tačnost: 0.6375956284153006
Trening podaci: Epoha 40: Mera gubitka: 2.424243688583374, Tačnost: 0.6898360655737705
Trening podaci: Epoha 50: Mera gubitka: 2.0617406368255615, Tačnost: 0.7370491803278688


In [37]:
###########################################################################################
# Evaluacija
###########################################################################################

# Učitavanje test skupa podataka
test_images, test_labels = load_data(test_data_dir)

# Predobrada slika, identično kao i za trening skup podataka.
test_scaled_imgs = [skimage.transform.resize(image, (32, 32), mode='constant')
                 for image in test_images]
test_labels_a = np.array(test_labels)
test_images_a = np.array(test_scaled_imgs)

# Pokretanje inferencije nad celim testnim skupom slika
predicted = session.run([predicted_classes], 
                        feed_dict={images_ph: test_scaled_imgs})[0]

test_loss_value = session.run(loss, 
                        feed_dict={images_ph: test_images_a, labels_ph: test_labels_a})

# Izračunavanje tačnosti
match_count = sum([int(y == y_) for y, y_ in zip(test_labels, predicted)])
test_accuracy = match_count / len(test_labels)
print("Test podaci: Mera gubitka: {0}, Tačnost: {1}".format(test_loss_value, test_accuracy))

session.close()

Test podaci: Mera gubitka: 1.4853664636611938, Tačnost: 0.6523809523809524
